<a href="https://colab.research.google.com/github/mmxMichelle/Python-AI/blob/main/Lab_5_HITS_and_SimRank_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 5 - HITS and SimRank

In this lab, you will practice the following exercises to enhance your skills in network analysis:

*  **[Exercise 1]** How to compute authority and hub scores of the HITS algorithm, using two different methods.

*  **[Exercise 2]** How to compute single-pair and all-pairs SimRank similarities, using recursive and matrix-based approaches, respectively.


# 1. HITS

The Hyperlink-Induced Topic Search (HITS) algorithm, developed by Jon Kleinberg, is a PageRank-like link analysis algorithm that evaluates the importance of nodes in a graph. The key idea behind HITS is the reinforcement assumption between two types of nodes:

*   **Authorities:** Nodes that are pointed to by many other nodes, indicating that they are good sources of information.
*   **Hubs:** Nodes that point to many other nodes, indicating that they are good at directing users to relevant information.

Let $A$ be the adjacency matrix of the graph. Let $a$ and $h$ be the authority vector and hub vector, respectively. The authority and hub scores mutually reinforce each other through a bi-recursive relationship, encapsulated by the following expressions:

$$
a=\frac{A^Th}{\|A^Th\|_2} \tag{1}
$$
$$
h=\frac{Aa}{\|Aa\|_2}     \tag{2}
$$


## Exercise 1.1

To efficiently compute the authority vector $a$ and hub vector $h$ in Eqs.(1) and (2), we substitute (2) into (1), which yields

$$
a=\frac{(A^TA)a}{\|(A^TA)a\|_2} \tag{3}
$$

This suggests $a$ is the dominant eigenvector of the matrix $(A^TA)$.

Similarly, substituting (1) into (2) produces

$$
h=\frac{(AA^T)h}{\|(AA^T)h\|_2} \tag{4}
$$

This suggests $h$ is the dominant eigenvector of the matrix $(AA^T)$.

Using the above method, write a piece of code to compute the authority vector $a$ and hub vector $h$ for the following graph:

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

# Create a directed graph G
G = nx.DiGraph()
G.add_edges_from([(1, 1), (1, 2), (2, 2), (2, 3), (3, 2)])

# Get the adjacency matrix A
A = nx.adjacency_matrix(G).toarray()

# Find eigenvalues (eigvals_a) and eigenvectors (eigvecs_a) for matrix (A'*A)
# Find eigenvalues (eigvals_h) and eigenvectors (eigvecs_h) for matrix (A*A')
# Hint: use np.linalg.eig() for eigenvalue decomposition
eigvals_a, eigvecs_a = np.linalg.eig(A.T @ A)
eigvals_h, eigvecs_h = np.linalg.eig(A @ A.T)

# Find the index (idx_a) of the dominant eigenvalue (largest magnitude) for eigvals_a
# Find the index (idx_h) of the dominant eigenvalue (largest magnitude) for eigvals_h
# Hint: use np.argmax() and np.abs()
idx_a = np.argmax(np.abs(eigvals_a))
idx_h = np.argmax(np.abs(eigvals_h))

# Get the dominant eigenvector (auth) by extracting the eigenvectors (eigvecs_a) as authority vector
# Get the dominant eigenvector (hub) by extracting the eigenvectors (eigvecs_h) as hub vector
auth = np.abs(eigvecs_a[:, idx_a])
hub = np.abs(eigvecs_h[:, idx_h])

# Normalise authority vector (auth) and hub vector (hub), respectively, using L2-norm
# Hint: use np.linalg.norm() to get L2-norm of a vector
auth /= np.linalg.norm(auth)
hub /= np.linalg.norm(hub)

# Display the results
print("Authority Vector:")
print(auth)
print("Hub Vector:")
print(hub)

Authority Vector:
[0.32505758 0.88807383 0.32505758]
Hub Vector:
[0.62796303 0.62796303 0.45970084]


##  Exercise 1.2

The Singular Value Decomposition (SVD) of a matrix $A$ also provides a more efficient way to compute the dominant eigenvector of both $(A^TA)$ and $(AA^T)$. Here is how we can use SVD to achieve this:

Let $A$ be the matrix with SVD $A=U\Sigma V^T$, where $U$ and $V$ are orthogonal matrices, and $\Sigma$ is a diagonal matrix with singular values on the diagonal. We have the following results:

*   The dominant eigenvector of $(AA^T)$, i.e., the hub vector $h$, is the column of $U$ corresponding to the largest singular value.

*   The dominant eigenvector of $(A^TA)$, i.e., the authority vector $a$, is the column of $V$ corresponding to the largest singular value.

Using the above SVD method, write a piece of code to compute the authority vector  $a$ and hub vector $h$ for the following graph.

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

# Create a directed graph G
G = nx.DiGraph()
G.add_edges_from([(1, 1), (1, 2), (2, 2), (2, 3), (3, 2)])

# Get the adjacency matrix A
A = nx.adjacency_matrix(G).toarray()

# Get the Singular Value Decomposition (SVD) of matrix A such that A = U*S*Vt
# Hint: use np.linalg.svd() function
U, S, Vt = np.linalg.svd(A)

#  Extract the dominant eigenvector corresponds to the column of (U)
#  with the highest singular value as the hub vector (hub)
hub = U[:, 0]

#  Extract the dominant eigenvector corresponds to the column of (V)
#  with the highest singular value as the authority vector (auth)
auth = Vt.T[:, 0]

# Normalise the authority and hub vectors, respectively, by dividing each vector by its L2 norm.
# Hint: use np.linalg.norm() function
hub /= np.linalg.norm(hub)
auth /= np.linalg.norm(auth)

# Print the result
print("Authority Vector:")
print(auth)

print("Hub Vector:")
print(hub)

Authority Vector:
[0.32505758 0.88807383 0.32505758]
Hub Vector:
[0.62796303 0.62796303 0.45970084]


# 2.  SimRank

SimRank is a similarity measure that assesses the similarity between two nodes in a graph. The fundamental idea behind SimRank is that two nodes are similar if their in-neighbors are also similar.

Let $s(a, b)$ be the SimRank similarity between two nodes $a$ and $b$. Let $I(a)$ denote the in-neighbor set of node $a$. Then, the recursive equation for $s(a, b)$ is defined as follows:

*   If $a = b$, then $s(a, b)=1$.
*   If $I(a) = \emptyset$ or $I(b) = \emptyset$, then $s(a, b)=0$.
*   Otherwise,
$$
s(a, b) = \frac{C}{\left|I(a)\right| \left|I(b)\right|}
 \sum_{i \in I(a)}\sum_{j \in I(b)}
 s(i, j)  \tag{5}
$$
where $C$ is a constant between $0$ and $1$, which is often set to $0.6$.


## Exercise 2.1

Write a function `simrank(G, a, b, C)` to recursively compute the SimRank similarity between nodes `a` and `b` in a digraph `G`  based on Eq.(5).

In [ ]:
import networkx as nx

def simrank(G, a, b, C=0.6):
    """
    Recursively compute SimRank similarity between nodes a and b in a directed graph G.

    Parameters:
    - G     : A digraph.
    - a, b  : Nodes for which to compute SimRank similarity.
    - C     : A constant decay factor, defaults to 0.6.

    Returns:
    - float: SimRank similarity score s(a,b) between a and b.
    """

    # Base case: If nodes are the same, similarity is 1.0
    if a == b:
        return 1.0

    # Get the in-neighbors of nodes a and b
    in_nbrs_a = set(G.predecessors(a))
    in_nbrs_b = set(G.predecessors(b))

    # Check if either set of in-neighbors is empty
    if not bool(in_nbrs_a) or not bool(in_nbrs_b):
        return 0.0

    # Initialize the similarity score
    s = 0.0

    # Iterate over pairs of in-neighbors and accumulate similarity scores
    for j in in_nbrs_b:
        for i in in_nbrs_a:
            s += simrank(G, i, j, C)

    # Compute and return the final similarity score
    return C / (G.in_degree(a) * G.in_degree(b)) * s


# Create a directed graph
G = nx.DiGraph()
G.add_edges_from([(1, 2), (1, 3), (1, 4), (2, 5), (2, 6), (3, 6), (3, 7), (4, 7)])

# Compute SimRank similarity for nodes 5 and 6
sim = simrank(G, 5, 6)

# Print the result
print(f"SimRank Similarity s(5,6) = {sim}")


SimRank Similarity s(5,6) = 0.48


## Exercise 2.2

In matrix notations, the SimRank Eq.(5) can also be rewritten as follows:

$$
S=\max \{C\cdot Q^T S Q, I \}
$$

where $Q$ is obtained by normalising each column of the adjacency matrix $A$. $I$ is an identity matrix, and $S$ is the similarity matrix, whose $(a,b)$-entry denotes the SimRank similarity between nodes $a$ and $b$.

To obtain the SimRank matrix $S$, we can use the following iterative method to compute $S_k$:

$$
S_k = \max \{C\cdot Q^T S_{k-1} Q, I \}  \quad \text{ with } \quad S_0=I
$$

Write a piece of code to iteratively compute the SimRank matrix:


In [ ]:
# Create a directed graph (G) using NetworkX
G = nx.DiGraph()
G.add_edges_from([(1, 2), (1, 3), (1, 4), (2, 5), (2, 6), (3, 6), (3, 7), (4, 7)])
A = nx.adjacency_matrix(G).toarray()

C = 0.6    # Damping factor
kmax = 5   # Maximum number of iterations

# Get the number of nodes (n) in the graph G
n = nx.number_of_nodes(G)

# Column-normalise the adjacency matrix A
col_sums = A.sum(axis=0)
col_sums[col_sums==0] = 1
Q = A / col_sums

# Initialise the similarity matrix with the identity matrix
S = np.identity(n)

# Perform SimRank iterations to update the similarity matrix
for k in range(kmax):
    S = C* (Q.T @ S @ Q)
    S = np.maximum(S, np.identity(n))

# Print the final similarity matrix after the specified number of iterations (kmax = 5)
print("SimRank Matrix S = ")
print(S)

SimRank Matrix S = 
[[1.   0.   0.   0.   0.   0.   0.  ]
 [0.   1.   0.6  0.6  0.   0.   0.  ]
 [0.   0.6  1.   0.6  0.   0.   0.  ]
 [0.   0.6  0.6  1.   0.   0.   0.  ]
 [0.   0.   0.   0.   1.   0.48 0.36]
 [0.   0.   0.   0.   0.48 1.   0.42]
 [0.   0.   0.   0.   0.36 0.42 1.  ]]
